In [1]:
# following from workbook 'TidyTextMining'
# trying to build an ngram tree of a given depth.
# could I just build it for full depth?

In [2]:
n = 3 #trigrams

In [4]:
library(dplyr)
library(tidytext)
library(tidyr)
library(ggplot2)
library(igraph)
library(ggraph)

In [5]:
enUsTwitter10kSubset <- readRDS(file='../data/subsets/enUsTwitter10kSubset.rds')
head(enUsTwitter10kSubset)

[[1]]
[1] "These late night walks are still religion to me. Thank goodness for these."

[[2]]
[1] "Enter to win an Orioles vintage cap by email or tweet the answer to: Which 2 O's pitchers hold the record for the most opening day starts?"

[[3]]
[1] "A Monthly payment plan is now available for those wishing to enroll in Hawaii's #1 RE Investor training program - the Millionaire Mentor..."

[[4]]
[1] "Game time!"

[[5]]
[1] "I feared that this was bound to happen Sugarland almost died 2 who should they sue The State of Indiana? whynot"

[[6]]
[1] "ADDY Awards tonight...plus giving scholarships to 5 deserving students...and something from our young advertising professionals"

In [9]:
textDf <- tibble(
    line = 1:length(enUsTwitter10kSubset), 
    text = unlist(enUsTwitter10kSubset)
)
nGramsDf <- textDf %>%
  unnest_tokens(ngram, text, token="ngrams", n=n)
head(nGramsDf)

line,ngram
<int>,<chr>
1,these late night
1,late night walks
1,night walks are
1,walks are still
1,are still religion
1,still religion to


In [15]:
nGramsSeparated <- nGramsDf %>%
  separate(ngram, as.character(1:n), sep = " ") #%>%
#   filter(!word1 %in% stop_words$word) %>%
#   filter(!word2 %in% stop_words$word) %>%

# # bigramsDfNoStop <- bigrams_filtered %>%
# #   unite(bigram, word1, word2, sep = " ")

# # new bigram counts:
# bigram_counts <- bigrams_filtered %>% 
#   count(word1, word2, sort = TRUE)

# head(bigram_counts)

In [21]:
head(nGramsSeparated)

line,1,2,3
<int>,<chr>,<chr>,<chr>
1,these,late,night
1,late,night,walks
1,night,walks,are
1,walks,are,still
1,are,still,religion
1,still,religion,to


In [33]:
nGramsDf %>% 
    mutate(containsStop = apply(nGramsSeparated[1:10,(1:n)-1], 1, print))

   line       1 
    "1" "these" 
  line      1 
   "1" "late" 
   line       1 
    "1" "night" 
   line       1 
    "1" "walks" 
 line     1 
  "1" "are" 
   line       1 
    "1" "still" 
      line          1 
       "1" "religion" 
line    1 
 "1" "to" 
line    1 
 "1" "me" 
   line       1 
    "1" "thank" 


ERROR: Error: Column `containsStop` must be length 107556 (the number of rows) or one, not 20


In [39]:
apply(nGramsSeparated[1:10,(1:n)+1], 1, print)

      1       2       3 
"these"  "late" "night" 
      1       2       3 
 "late" "night" "walks" 
      1       2       3 
"night" "walks"   "are" 
      1       2       3 
"walks"   "are" "still" 
         1          2          3 
     "are"    "still" "religion" 
         1          2          3 
   "still" "religion"       "to" 
         1          2          3 
"religion"       "to"       "me" 
      1       2       3 
   "to"    "me" "thank" 
         1          2          3 
      "me"    "thank" "goodness" 
         1          2          3 
   "thank" "goodness"      "for" 


these,late,night,walks,are,still,religion,to,me,thank
late,night,walks,are,still,religion,to,me,thank,goodness
night,walks,are,still,religion,to,me,thank,goodness,for


In [40]:
nGramsSeparated[1:10,(1:n)+1]

1,2,3
<chr>,<chr>,<chr>
these,late,night
late,night,walks
night,walks,are
walks,are,still
are,still,religion
still,religion,to
religion,to,me
to,me,thank
me,thank,goodness


In [41]:
# different approach:
# tokenise into single line df (include EOL tokens)
# filter (stop words? profanity, etc)
# add these words to graph
# shift by one, add word1 -> word2 edges to graph
# repeat to level n or until 
#<ABANDONED>

In [43]:
head(nGramsSeparated)

line,1,2,3
<int>,<chr>,<chr>,<chr>
1,these,late,night
1,late,night,walks
1,night,walks,are
1,walks,are,still
1,are,still,religion
1,still,religion,to


In [58]:
library(treemap)
library(data.tree)

data(GNI2014)
GNI2014$pathString <- paste("world", 
                            GNI2014$continent, 
                            GNI2014$country, 
                            sep = "/")
head(GNI2014)
population <- as.Node(GNI2014)
print(population, "iso3", "population", "GNI", limit = 20)

,iso3,country,continent,population,GNI,pathString
,<chr>,<chr>,<fct>,<dbl>,<int>,<chr>
3,BMU,Bermuda,North America,67837,106140,world/North America/Bermuda
4,NOR,Norway,Europe,4676305,103630,world/Europe/Norway
5,QAT,Qatar,Asia,833285,92200,world/Asia/Qatar
6,CHE,Switzerland,Europe,7604467,88120,world/Europe/Switzerland
7,MAC,"Macao SAR, China",Asia,559846,76270,"world/Asia/Macao SAR, China"
8,LUX,Luxembourg,Europe,491775,75990,world/Europe/Luxembourg


                                levelName iso3 population    GNI
1  world                                               NA     NA
2   ¦--North America                                   NA     NA
3   ¦   ¦--Bermuda                         BMU      67837 106140
4   ¦   ¦--United States                   USA  313973000  55200
5   ¦   ¦--Canada                          CAN   33487208  51630
6   ¦   ¦--Bahamas, The                    BHS     309156  20980
7   ¦   ¦--Trinidad and Tobago             TTO    1310000  20070
8   ¦   ¦--Puerto Rico                     PRI    3971020  19310
9   ¦   ¦--Barbados                        BRB     284589  15310
10  ¦   ¦--St. Kitts and Nevis             KNA      40131  14920
11  ¦   ¦--Antigua and Barbuda             ATG      85632  13300
12  ¦   ¦--Panama                          PAN    3360474  11130
13  ¦   ¦--Costa Rica                      CRI    4253877  10120
14  ¦   ¦--Mexico                          MEX  111211789   9870
15  ¦   ¦--Grenada       

In [62]:
nGramsSeparated$paths <- apply(nGramsSeparated[,(1:n)+1], 1, paste, collapse="/")
as.Node(nGramsSeparated)

ERROR: Error in `[[<-.data.frame`(`*tmp*`, pathName, value = character(0)): replacement has 0 rows, data has 107556


In [64]:
# apply(nGramsSeparated[,(1:n)+1], 1, paste, collapse="/")

In [77]:
nGramsSeparatedShort <- head(nGramsSeparated)

toPath <- function(row){
    paste(c("<S>", paste(row, collapse = "/"), "<E>"),collapse = "/" )
}

nGramsSeparated$pathString <-apply(nGramsSeparated[,(1:n)+1], 1, toPath)
as.Node(nGramsSeparated)

              levelName
1  <S>                 
2   ¦--these           
3   ¦   °--late        
4   ¦       °--night   
5   ¦           °--<E> 
6   ¦--late            
7   ¦   °--night       
8   ¦       °--walks   
9   ¦           °--<E> 
10  ¦--night           
11  ¦   °--walks       
12  ¦       °--are     
13  ¦           °--<E> 
14  ¦--walks           
15  ¦   °--are         
16  ¦       °--still   
17  ¦           °--<E> 
18  ¦--are             
19  ¦   °--still       
20  ¦       °--religion
21  ¦           °--<E> 
22  °--still           
23      °--religion    
24          °--to      
25              °--<E> 

In [78]:
nGramsSeparatedShort <- head(nGramsSeparated)

toPath <- function(row){
    paste(c("<S>", paste(row, collapse = "/"), "<E>"),collapse = "/" )
}

nGramsSeparated$pathString <-apply(nGramsSeparated[,(1:n)+1], 1, toPath)
as.Node(nGramsSeparated)

Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'children

“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'position' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'position2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONS

“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'children' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'children2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'children' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'ch

“Name 'root' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'root2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'root' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'root2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'root' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'root2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'count' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'count2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'count' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'count2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'count' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'count2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'children' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'chil

“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'name' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'name2' instead.”Warning message in CheckNameReservedWord(path, check):
“Name 'count' is a reserved word as defined in NODE_RESERVED_NAMES_CONST. Using 'count2' inste

                            levelName
1   <S>                              
2    ¦--these                        
3    ¦   ¦--late                     
4    ¦   ¦   °--night                
5    ¦   ¦       °--<E>              
6    ¦   ¦--mfs                      
7    ¦   ¦   °--than                 
8    ¦   ¦       °--<E>              
9    ¦   ¦--events                   
10   ¦   ¦   °--get                  
11   ¦   ¦       °--<E>              
12   ¦   ¦--parties                  
13   ¦   ¦   °--started              
14   ¦   ¦       °--<E>              
15   ¦   ¦--amazing                  
16   ¦   ¦   °--bands                
17   ¦   ¦       °--<E>              
18   ¦   ¦--3                        
19   ¦   ¦   °--songs                
20   ¦   ¦       °--<E>              
21   ¦   ¦--last                     
22   ¦   ¦   °--few                  
23   ¦   ¦       °--<E>              
24   ¦   ¦--asshats                  
25   ¦   ¦   °--at                   
26   ¦   ¦  